In [44]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_validate, ShuffleSplit, KFold, cross_val_score, GridSearchCV

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


# Importing & Exploring Data

In [45]:
df_crashes = pd.read_csv('data/Traffic_Crashes_-_Crashes.csv')
df_crashes.head()

,CRASH_RECORD_ID,RD_NO,CRASH_DATE_EST_I,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,LANE_CNT,ALIGNMENT,ROADWAY_SURFACE_COND,ROAD_DEFECT,REPORT_TYPE,CRASH_TYPE,INTERSECTION_RELATED_I,NOT_RIGHT_OF_WAY_I,HIT_AND_RUN_I,DAMAGE,DATE_POLICE_NOTIFIED,PRIM_CONTRIBUTORY_CAUSE,SEC_CONTRIBUTORY_CAUSE,STREET_NO,STREET_DIRECTION,STREET_NAME,BEAT_OF_OCCURRENCE,PHOTOS_TAKEN_I,STATEMENTS_TAKEN_I,DOORING_I,WORK_ZONE_I,WORK_ZONE_TYPE,WORKERS_PRESENT_I,NUM_UNITS,MOST_SEVERE_INJURY,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION
0,062f5a6f6b87b762165d4da04d6d3a181385776a10b051...,JF378246,NaN,08/31/2022 10:13:00 PM,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",SIDESWIPE SAME DIRECTION,DIVIDED - W/MEDIAN BARRIER,NaN,STRAIGHT AND LEVEL,DRY,NO DEFECTS,ON SCENE,INJURY AND / OR TOW DUE TO CRASH,NaN,NaN,Y,"OVER $1,500",08/31/2022 10:24:00 PM,UNABLE TO DETERMINE,UNABLE TO DETERMINE,4315,N,CICERO AVE,1722.0,NaN,Y,NaN,NaN,NaN,NaN,4,NONINCAPACITATING INJURY,2.0,0.0,0.0,2.0,0.0,1.0,0.0,22,4,8,41.959389,-87.747348,POINT (-87.747347903299 41.959389159692)
1,0115ade9a755e835255508463f7e9c4a9a0b47e9304238...,JF318029,NaN,07/15/2022 12:45:00 AM,30,UNKNOWN,UNKNOWN,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,NOT DIVIDED,NaN,STRAIGHT AND LEVEL,DRY,UNKNOWN,ON SCENE,NO INJURY / DRIVE AWAY,NaN,NaN,NaN,"OVER $1,500",07/15/2022 12:50:00 AM,UNABLE TO DETERMINE,UNABLE TO DETERMINE,300,N,CENTRAL PARK AVE,1123.0,NaN,NaN,NaN,NaN,NaN,NaN,2,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0,6,7,41.886336,-87.716203,POINT (-87.716203130599 41.886336409761)
2,017040c61958d2fa977c956b2bd2d6759ef7754496dc96...,JF324552,NaN,07/15/2022 06:50:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,NOT DIVIDED,NaN,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NOT ON SCENE (DESK REPORT),NO INJURY / DRIVE AWAY,NaN,NaN,NaN,"OVER $1,500",07/20/2022 11:00:00 AM,UNABLE TO DETERMINE,UNABLE TO DETERMINE,2359,N,ASHLAND AVE,1811.0,NaN,NaN,NaN,NaN,NaN,NaN,2,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,2.0,0.0,18,6,7,41.925111,-87.667997,POINT (-87.667997321599 41.925110815832)
3,01aaa759c6bbefd0f584226fbd88bdc549de3ed1e46255...,JF319819,NaN,07/15/2022 05:10:00 PM,40,NO CONTROLS,NO CONTROLS,CLOUDY/OVERCAST,DAYLIGHT,ANGLE,DIVIDED - W/MEDIAN BARRIER,NaN,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NOT ON SCENE (DESK REPORT),NO INJURY / DRIVE AWAY,NaN,NaN,NaN,"OVER $1,500",07/16/2022 03:20:00 PM,IMPROPER LANE USAGE,NOT APPLICABLE,5100,N,LAKE SHORE DR NB,2024.0,NaN,NaN,NaN,NaN,NaN,NaN,2,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,2.0,0.0,17,6,7,41.975826,-87.650420,POINT (-87.650419778017 41.975826016449)
4,04f21d51f8189e34abf37c7973607fa076965d216b514f...,JC366684,NaN,07/22/2019 12:00:00 PM,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PEDALCYCLIST,NOT DIVIDED,NaN,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NaN,INJURY AND / OR TOW DUE TO CRASH,NaN,NaN,Y,"$501 - $1,500",07/27/2019 11:30:00 AM,UNABLE TO DETERMINE,UNABLE TO DETERMINE,2500,E,87TH ST,423.0,NaN,NaN,N,NaN,NaN,NaN,2,INCAPACITATING INJURY,1.0,0.0,1.0,0.0,0.0,1.0,0.0,12,2,7,41.737337,-87.563560,POINT (-87.563560484875 41.737337119112)


In [46]:
df_crashes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 655182 entries, 0 to 655181
Data columns (total 49 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   CRASH_RECORD_ID                655182 non-null  object 
 1   RD_NO                          650595 non-null  object 
 2   CRASH_DATE_EST_I               49695 non-null   object 
 3   CRASH_DATE                     655182 non-null  object 
 4   POSTED_SPEED_LIMIT             655182 non-null  int64  
 5   TRAFFIC_CONTROL_DEVICE         655182 non-null  object 
 6   DEVICE_CONDITION               655182 non-null  object 
 7   WEATHER_CONDITION              655182 non-null  object 
 8   LIGHTING_CONDITION             655182 non-null  object 
 9   FIRST_CRASH_TYPE               655182 non-null  object 
 10  TRAFFICWAY_TYPE                655182 non-null  object 
 11  LANE_CNT                       198991 non-null  float64
 12  ALIGNMENT                     

In [47]:
df_crashes.isna().sum()

CRASH_RECORD_ID                       0
RD_NO                              4587
CRASH_DATE_EST_I                 605487
CRASH_DATE                            0
POSTED_SPEED_LIMIT                    0
TRAFFIC_CONTROL_DEVICE                0
DEVICE_CONDITION                      0
WEATHER_CONDITION                     0
LIGHTING_CONDITION                    0
FIRST_CRASH_TYPE                      0
TRAFFICWAY_TYPE                       0
LANE_CNT                         456191
ALIGNMENT                             0
ROADWAY_SURFACE_COND                  0
ROAD_DEFECT                           0
REPORT_TYPE                       17491
CRASH_TYPE                            0
INTERSECTION_RELATED_I           505314
NOT_RIGHT_OF_WAY_I               624336
HIT_AND_RUN_I                    453020
DAMAGE                                0
DATE_POLICE_NOTIFIED                  0
PRIM_CONTRIBUTORY_CAUSE               0
SEC_CONTRIBUTORY_CAUSE                0
STREET_NO                             0


In [48]:
# Keeping relevant Features
df_crashes_drop = df_crashes [[
    'CRASH_RECORD_ID',
    'RD_NO',
    'CRASH_DATE',
    'POSTED_SPEED_LIMIT',
    'WEATHER_CONDITION',
    'LIGHTING_CONDITION',
    'FIRST_CRASH_TYPE',
    'ROADWAY_SURFACE_COND',
    'ROAD_DEFECT',
    'CRASH_TYPE',
    'DAMAGE',
    'PRIM_CONTRIBUTORY_CAUSE',
    'STREET_NAME',
    'NUM_UNITS',
    'INJURIES_TOTAL',
    'INJURIES_FATAL',
    'CRASH_HOUR',
    'CRASH_DAY_OF_WEEK',
    'CRASH_MONTH',
    'LATITUDE',
    'LONGITUDE',
#    'LOCATION',
]]
print(df_crashes_drop.shape)

(655182, 21)


In [49]:
df_crashes_drop.head(30)

,CRASH_RECORD_ID,RD_NO,CRASH_DATE,POSTED_SPEED_LIMIT,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,ROADWAY_SURFACE_COND,ROAD_DEFECT,CRASH_TYPE,DAMAGE,PRIM_CONTRIBUTORY_CAUSE,STREET_NAME,NUM_UNITS,INJURIES_TOTAL,INJURIES_FATAL,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE
0,062f5a6f6b87b762165d4da04d6d3a181385776a10b051...,JF378246,08/31/2022 10:13:00 PM,30,CLEAR,"DARKNESS, LIGHTED ROAD",SIDESWIPE SAME DIRECTION,DRY,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH,"OVER $1,500",UNABLE TO DETERMINE,CICERO AVE,4,2.0,0.0,22,4,8,41.959389,-87.747348
1,0115ade9a755e835255508463f7e9c4a9a0b47e9304238...,JF318029,07/15/2022 12:45:00 AM,30,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,DRY,UNKNOWN,NO INJURY / DRIVE AWAY,"OVER $1,500",UNABLE TO DETERMINE,CENTRAL PARK AVE,2,0.0,0.0,0,6,7,41.886336,-87.716203
2,017040c61958d2fa977c956b2bd2d6759ef7754496dc96...,JF324552,07/15/2022 06:50:00 PM,30,CLEAR,DAYLIGHT,REAR END,DRY,NO DEFECTS,NO INJURY / DRIVE AWAY,"OVER $1,500",UNABLE TO DETERMINE,ASHLAND AVE,2,0.0,0.0,18,6,7,41.925111,-87.667997
3,01aaa759c6bbefd0f584226fbd88bdc549de3ed1e46255...,JF319819,07/15/2022 05:10:00 PM,40,CLOUDY/OVERCAST,DAYLIGHT,ANGLE,DRY,NO DEFECTS,NO INJURY / DRIVE AWAY,"OVER $1,500",IMPROPER LANE USAGE,LAKE SHORE DR NB,2,0.0,0.0,17,6,7,41.975826,-87.650420
4,04f21d51f8189e34abf37c7973607fa076965d216b514f...,JC366684,07/22/2019 12:00:00 PM,30,CLEAR,DAYLIGHT,PEDALCYCLIST,DRY,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH,"$501 - $1,500",UNABLE TO DETERMINE,87TH ST,2,1.0,0.0,12,2,7,41.737337,-87.563560
5,0362b3f8ddb8589dfe889356efbc2ad213efe2031556f0...,JF318271,07/15/2022 08:44:00 AM,30,RAIN,DAYLIGHT,REAR END,WET,NO DEFECTS,NO INJURY / DRIVE AWAY,"OVER $1,500",UNABLE TO DETERMINE,CICERO AVE,2,0.0,0.0,8,6,7,41.944199,-87.747157
6,128ced0ce1c4d589766fcda0c30b8158c610b1c304ede6...,JF385203,08/31/2022 09:00:00 AM,10,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,DRY,NO DEFECTS,NO INJURY / DRIVE AWAY,$500 OR LESS,UNABLE TO DETERMINE,47TH ST,2,0.0,0.0,9,4,8,41.807712,-87.744440
7,03d3679ef44bb8aa0f2060cb0376f3eeb1d9dbb2197322...,JF318956,07/15/2022 04:10:00 PM,30,RAIN,DAYLIGHT,PEDESTRIAN,WET,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH,$500 OR LESS,FAILING TO YIELD RIGHT-OF-WAY,LAFLIN ST,2,1.0,0.0,16,6,7,41.855974,-87.663860
8,12acc41aaa2bdc2be15740de5c3c6cf59fca238e905d11...,JF377463,08/31/2022 11:50:00 AM,30,CLEAR,DAYLIGHT,FIXED OBJECT,DRY,NO DEFECTS,NO INJURY / DRIVE AWAY,"OVER $1,500",PHYSICAL CONDITION OF DRIVER,91ST ST,1,0.0,0.0,11,4,8,41.730216,-87.548387
9,0e44b481f5433e93a5a069f44ce4c79cfdad30ee4191c3...,JC402842,08/22/2019 03:37:00 PM,30,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,DRY,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH,"OVER $1,500",UNABLE TO DETERMINE,BERTEAU AVE,2,2.0,0.0,15,5,8,41.956477,-87.785397


In [50]:
df_crashes_drop.describe()

,POSTED_SPEED_LIMIT,NUM_UNITS,INJURIES_TOTAL,INJURIES_FATAL,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE
count,655182.000000,655182.000000,653792.000000,653792.000000,655182.000000,655182.000000,655182.000000,651108.000000,651108.000000
mean,28.356959,2.034471,0.184946,0.001169,13.222685,4.124272,6.641816,41.854503,-87.673305
std,6.296888,0.451588,0.558173,0.037166,5.549903,1.980657,3.365989,0.331127,0.672390
min,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,-87.936193
25%,30.000000,2.000000,0.000000,0.000000,9.000000,2.000000,4.000000,41.781166,-87.721366
50%,30.000000,2.000000,0.000000,0.000000,14.000000,4.000000,7.000000,41.874533,-87.673586
75%,30.000000,2.000000,0.000000,0.000000,17.000000,6.000000,9.000000,41.923996,-87.632948
max,99.000000,18.000000,21.000000,4.000000,23.000000,7.000000,12.000000,42.022780,0.000000


In [51]:
# Dropping Rows with Latitude & Longitude = 0
df_crashes_drop = df_crashes_drop[df_crashes_drop['LATITUDE'] != 0]

In [52]:
df_crashes_drop.isna().sum()

CRASH_RECORD_ID               0
RD_NO                      4587
CRASH_DATE                    0
POSTED_SPEED_LIMIT            0
WEATHER_CONDITION             0
LIGHTING_CONDITION            0
FIRST_CRASH_TYPE              0
ROADWAY_SURFACE_COND          0
ROAD_DEFECT                   0
CRASH_TYPE                    0
DAMAGE                        0
PRIM_CONTRIBUTORY_CAUSE       0
STREET_NAME                   1
NUM_UNITS                     0
INJURIES_TOTAL             1390
INJURIES_FATAL             1390
CRASH_HOUR                    0
CRASH_DAY_OF_WEEK             0
CRASH_MONTH                   0
LATITUDE                   4074
LONGITUDE                  4074
dtype: int64

In [53]:
#df_crashes_drop.fillna('UNKOWN', inplace=True)
#df_crashes_drop.isna().sum()

In [54]:
df_crashes_drop.replace({'UNKNOWN':np.nan}, inplace=True)

In [55]:
df_vehicles = pd.read_csv('data/Traffic_Crashes_-_Vehicles.csv')
df_vehicles.head()


C:\Users\milad\Documents\Flatiron\Anaconda\envs\learn-env\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (19,21,40,41,42,44,48,49,50,53,55,58,59,61,71) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,CRASH_UNIT_ID,CRASH_RECORD_ID,RD_NO,CRASH_DATE,UNIT_NO,UNIT_TYPE,NUM_PASSENGERS,VEHICLE_ID,CMRC_VEH_I,MAKE,MODEL,LIC_PLATE_STATE,VEHICLE_YEAR,VEHICLE_DEFECT,VEHICLE_TYPE,VEHICLE_USE,TRAVEL_DIRECTION,MANEUVER,TOWED_I,FIRE_I,OCCUPANT_CNT,EXCEED_SPEED_LIMIT_I,TOWED_BY,TOWED_TO,AREA_00_I,AREA_01_I,AREA_02_I,AREA_03_I,AREA_04_I,AREA_05_I,AREA_06_I,AREA_07_I,AREA_08_I,AREA_09_I,AREA_10_I,AREA_11_I,AREA_12_I,AREA_99_I,FIRST_CONTACT_POINT,CMV_ID,USDOT_NO,CCMC_NO,ILCC_NO,COMMERCIAL_SRC,GVWR,CARRIER_NAME,CARRIER_STATE,CARRIER_CITY,HAZMAT_PLACARDS_I,HAZMAT_NAME,UN_NO,HAZMAT_PRESENT_I,HAZMAT_REPORT_I,HAZMAT_REPORT_NO,MCS_REPORT_I,MCS_REPORT_NO,HAZMAT_VIO_CAUSE_CRASH_I,MCS_VIO_CAUSE_CRASH_I,IDOT_PERMIT_NO,WIDE_LOAD_I,TRAILER1_WIDTH,TRAILER2_WIDTH,TRAILER1_LENGTH,TRAILER2_LENGTH,TOTAL_VEHICLE_LENGTH,AXLE_CNT,VEHICLE_CONFIG,CARGO_BODY_TYPE,LOAD_TYPE,HAZMAT_OUT_OF_SERVICE_I,MCS_OUT_OF_SERVICE_I,HAZMAT_CLASS
0,829999,24ddf9fd8542199d832e1c223cc474e5601b356f1d77a6...,JD124535,01/22/2020 06:25:00 AM,1,DRIVER,NaN,796949.0,NaN,INFINITI,UNKNOWN,IL,2017.0,NONE,PASSENGER,PERSONAL,N,STRAIGHT AHEAD,NaN,NaN,1.0,NaN,NaN,NaN,NaN,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FRONT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,749947,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,09/28/2019 03:30:00 AM,1,DRIVER,NaN,834816.0,NaN,HONDA,CIVIC,IL,2016.0,UNKNOWN,PASSENGER,PERSONAL,N,STRAIGHT AHEAD,NaN,NaN,1.0,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FRONT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,749949,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,09/28/2019 03:30:00 AM,2,PARKED,NaN,834819.0,NaN,TOYOTA,YARIS,IL,2010.0,NONE,UNKNOWN/NA,PERSONAL,N,PARKED,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,ROOF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,749950,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,09/28/2019 03:30:00 AM,3,PARKED,NaN,834817.0,NaN,GENERAL MOTORS CORPORATION (GMC),SIERRA,IL,2008.0,UNKNOWN,UNKNOWN/NA,UNKNOWN/NA,N,PARKED,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,ROOF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,871921,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,JD208731,04/13/2020 10:50:00 PM,2,DRIVER,NaN,827212.0,NaN,BUICK,ENCORE,IL,NaN,NONE,PASSENGER,PERSONAL,W,STRAIGHT AHEAD,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FRONT-RIGHT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
df_vehicles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1335230 entries, 0 to 1335229
Data columns (total 72 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   CRASH_UNIT_ID             1335230 non-null  int64  
 1   CRASH_RECORD_ID           1335230 non-null  object 
 2   RD_NO                     1325924 non-null  object 
 3   CRASH_DATE                1335230 non-null  object 
 4   UNIT_NO                   1335230 non-null  int64  
 5   UNIT_TYPE                 1333361 non-null  object 
 6   NUM_PASSENGERS            198353 non-null   float64
 7   VEHICLE_ID                1304967 non-null  float64
 8   CMRC_VEH_I                24927 non-null    object 
 9   MAKE                      1304962 non-null  object 
 10  MODEL                     1304818 non-null  object 
 11  LIC_PLATE_STATE           1187047 non-null  object 
 12  VEHICLE_YEAR              1092432 non-null  float64
 13  VEHICLE_DEFECT            1

In [57]:
# filling all missing values with UNKNOWN string, so we can encode later
#df_vehicles_w_occ.fillna('UNKOWN', inplace=True)
#df_vehicles_w_occ.isna().sum()


In [58]:
# Keeping Relevant Features
df_vehicles_drop = df_vehicles [[
    'CRASH_RECORD_ID',
#    'RD_NO',
#    'CRASH_DATE',
    'VEHICLE_ID',
    'MAKE',
    'MODEL',
    'LIC_PLATE_STATE',
    'VEHICLE_YEAR',
#    'VEHICLE_DEFECT',
#    'VEHICLE_TYPE',
#    'VEHICLE_USE',
    'TRAVEL_DIRECTION',
    'OCCUPANT_CNT',
#    'VEHICLE_CONFIG',
]]
print(df_vehicles_drop.shape)

(1335230, 8)


In [59]:
df_vehicles_drop.describe()

,VEHICLE_ID,VEHICLE_YEAR,OCCUPANT_CNT
count,1.304967e+06,1.092432e+06,1.304967e+06
mean,6.789888e+05,2.013626e+03,1.079589e+00
std,3.917114e+05,1.439261e+02,7.843770e-01
min,2.000000e+00,1.900000e+03,0.000000e+00
25%,3.400725e+05,2.006000e+03,1.000000e+00
50%,6.794450e+05,2.012000e+03,1.000000e+00
75%,1.017828e+06,2.016000e+03,1.000000e+00
max,1.358762e+06,9.999000e+03,9.900000e+01


In [60]:
# Dropping rows with vehicle year bigger than 2022 and below 1970
df_vehicles_drop1 = df_vehicles_drop[(df_vehicles_drop['VEHICLE_YEAR'] >= 1970) & (df_vehicles_drop['VEHICLE_YEAR'] <= 2022) | (df_vehicles_drop['VEHICLE_YEAR'].isna())]

In [61]:
df_vehicles_drop1.shape

(1333732, 8)

In [62]:
pd.set_option('display.max_rows', 100)
df_vehicles_drop1.isna().sum()

CRASH_RECORD_ID          0
VEHICLE_ID           30263
MAKE                 30268
MODEL                30411
LIC_PLATE_STATE     148025
VEHICLE_YEAR        242798
TRAVEL_DIRECTION     30263
OCCUPANT_CNT         30263
dtype: int64

In [63]:
pd.set_option('display.max_rows', 100)
df_vehicles_drop1.head(50)

,CRASH_RECORD_ID,VEHICLE_ID,MAKE,MODEL,LIC_PLATE_STATE,VEHICLE_YEAR,TRAVEL_DIRECTION,OCCUPANT_CNT
0,24ddf9fd8542199d832e1c223cc474e5601b356f1d77a6...,796949.0,INFINITI,UNKNOWN,IL,2017.0,N,1.0
1,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,834816.0,HONDA,CIVIC,IL,2016.0,N,1.0
2,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,834819.0,TOYOTA,YARIS,IL,2010.0,N,0.0
3,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,834817.0,GENERAL MOTORS CORPORATION (GMC),SIERRA,IL,2008.0,N,0.0
4,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,827212.0,BUICK,ENCORE,IL,NaN,W,1.0
5,80f5b6eff349a72094c9c7a11745549caa8bd39b57cedd...,453462.0,LINCOLN-CONTINENTAL,MKT,IL,2010.0,S,0.0
6,ecf9e646f89a73ed3df0f373e4fbd18e91f5a595004019...,792247.0,HONDA,CRV,IL,2014.0,W,1.0
7,ecf9e646f89a73ed3df0f373e4fbd18e91f5a595004019...,792252.0,SUBARU,FORESTER,IL,2018.0,S,2.0
8,c7c57a49274b3aa953d0b76346f68e116ed8c3d72a683a...,794648.0,UNKNOWN,UNKNOWN,IL,NaN,UNKNOWN,1.0
9,c7c57a49274b3aa953d0b76346f68e116ed8c3d72a683a...,794653.0,INFINITI,UNKNOWN,IL,2014.0,UNKNOWN,0.0


In [64]:
# Dropping Vehicles with 0 Occupant (Parked Cars)
df_vehicles_w_occ = df_vehicles_drop1[df_vehicles_drop.OCCUPANT_CNT != 0.0]
print(df_vehicles_w_occ.shape)

(1160416, 8)


<ipython-input-64-a0910bced00f>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_vehicles_w_occ = df_vehicles_drop1[df_vehicles_drop.OCCUPANT_CNT != 0.0]


In [65]:
df_vehicles_w_occ.isna().sum()

CRASH_RECORD_ID          0
VEHICLE_ID           30263
MAKE                 30268
MODEL                30390
LIC_PLATE_STATE     141601
VEHICLE_YEAR        231819
TRAVEL_DIRECTION     30263
OCCUPANT_CNT         30263
dtype: int64

In [66]:
# Replacing all UNKNOWN values to missing values
df_vehicles_w_occ.replace({'UNKNOWN':np.nan}, inplace=True)

C:\Users\milad\Documents\Flatiron\Anaconda\envs\learn-env\lib\site-packages\pandas\core\frame.py:4379: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [67]:
df_vehicles_w_occ.isna().sum()

CRASH_RECORD_ID          0
VEHICLE_ID           30263
MAKE                158018
MODEL               196867
LIC_PLATE_STATE     141601
VEHICLE_YEAR        231819
TRAVEL_DIRECTION    108390
OCCUPANT_CNT         30263
dtype: int64

In [68]:
df_people = pd.read_csv('data/Traffic_Crashes_-_PEOPLE.csv')
df_people.head()


C:\Users\milad\Documents\Flatiron\Anaconda\envs\learn-env\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (20,24,25,26,29) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,PERSON_ID,PERSON_TYPE,CRASH_RECORD_ID,RD_NO,VEHICLE_ID,CRASH_DATE,SEAT_NO,CITY,STATE,ZIPCODE,SEX,AGE,DRIVERS_LICENSE_STATE,DRIVERS_LICENSE_CLASS,SAFETY_EQUIPMENT,AIRBAG_DEPLOYED,EJECTION,INJURY_CLASSIFICATION,HOSPITAL,EMS_AGENCY,EMS_RUN_NO,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,PEDPEDAL_ACTION,PEDPEDAL_VISIBILITY,PEDPEDAL_LOCATION,BAC_RESULT,BAC_RESULT VALUE,CELL_PHONE_USE
0,O749947,DRIVER,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,834816.0,09/28/2019 03:30:00 AM,NaN,CHICAGO,IL,60651,M,25.0,IL,D,NONE PRESENT,DEPLOYMENT UNKNOWN,NONE,NO INDICATION OF INJURY,NaN,NaN,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
1,O871921,DRIVER,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,JD208731,827212.0,04/13/2020 10:50:00 PM,NaN,CHICAGO,IL,60620,M,37.0,IL,NaN,SAFETY BELT USED,DID NOT DEPLOY,NONE,NO INDICATION OF INJURY,NaN,NaN,NaN,NONE,NOT OBSCURED,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
2,O10018,DRIVER,71162af7bf22799b776547132ebf134b5b438dcf3dac6b...,HY484534,9579.0,11/01/2015 05:00:00 AM,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,USAGE UNKNOWN,DEPLOYMENT UNKNOWN,NONE,NO INDICATION OF INJURY,NaN,NaN,NaN,IMPROPER BACKING,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
3,O10038,DRIVER,c21c476e2ccc41af550b5d858d22aaac4ffc88745a1700...,HY484750,9598.0,11/01/2015 08:00:00 AM,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,USAGE UNKNOWN,DEPLOYMENT UNKNOWN,UNKNOWN,NO INDICATION OF INJURY,NaN,NaN,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
4,O10039,DRIVER,eb390a4c8e114c69488f5fb8a097fe629f5a92fd528cf4...,HY484778,9600.0,11/01/2015 10:15:00 AM,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,USAGE UNKNOWN,DEPLOYMENT UNKNOWN,UNKNOWN,NO INDICATION OF INJURY,NaN,NaN,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN


In [69]:
pd.set_option('display.max_rows', 100)
df_people.isna().sum()

PERSON_ID                      0
PERSON_TYPE                    0
CRASH_RECORD_ID                0
RD_NO                      10002
VEHICLE_ID                 28251
CRASH_DATE                     0
SEAT_NO                  1145710
CITY                      386612
STATE                     372988
ZIPCODE                   477431
SEX                        22243
AGE                       417101
DRIVERS_LICENSE_STATE     592164
DRIVERS_LICENSE_CLASS     719487
SAFETY_EQUIPMENT            4087
AIRBAG_DEPLOYED            27110
EJECTION                   17338
INJURY_CLASSIFICATION        632
HOSPITAL                 1186950
EMS_AGENCY               1281208
EMS_RUN_NO               1411946
DRIVER_ACTION             294521
DRIVER_VISION             294925
PHYSICAL_CONDITION        293716
PEDPEDAL_ACTION          1410885
PEDPEDAL_VISIBILITY      1410940
PEDPEDAL_LOCATION        1410889
BAC_RESULT                293506
BAC_RESULT VALUE         1435858
CELL_PHONE_USE           1436454
dtype: int

In [70]:
# Keeping Relevant Features
df_people_drop = df_people [[
    'CRASH_RECORD_ID',
#    'RD_NO',
    'PERSON_ID',
    'PERSON_TYPE',
    'VEHICLE_ID',
    'CITY',
    'STATE',
    'ZIPCODE',
    'SEX',
    'AGE',
    'DRIVERS_LICENSE_STATE',
#    'DRIVERS_LICENSE_CLASS',
    'SAFETY_EQUIPMENT',
    'AIRBAG_DEPLOYED',
#    'EJECTION',
    'INJURY_CLASSIFICATION',
#    'DRIVER_VISION',
    'DRIVER_ACTION',
    'PHYSICAL_CONDITION',
#    'PEDPEDAL_ACTION',
#    'PEDPEDAL_VISIBILITY',
#    'PEDPEDAL_LOCATION',
#    'BAC_RESULT',
#    'BAC_RESULT VALUE',
#    'CELL_PHONE_USE',
]]
print(df_people_drop.shape)

(1437611, 15)


In [71]:
# Filtering the data with Drivers only
df_people_driver = df_people_drop[df_people_drop.PERSON_TYPE == 'DRIVER']
print(df_people_driver.shape)

(1117947, 15)


In [72]:
df_people_driver.isna().sum()

CRASH_RECORD_ID               0
PERSON_ID                     0
PERSON_TYPE                   0
VEHICLE_ID                  632
CITY                     268192
STATE                    281819
ZIPCODE                  325989
SEX                          97
AGE                      299295
DRIVERS_LICENSE_STATE    272609
SAFETY_EQUIPMENT              0
AIRBAG_DEPLOYED               0
INJURY_CLASSIFICATION         0
DRIVER_ACTION                 0
PHYSICAL_CONDITION            0
dtype: int64

In [73]:
pd.set_option('display.max_columns', 100)
df_people_driver.head(20)

,CRASH_RECORD_ID,PERSON_ID,PERSON_TYPE,VEHICLE_ID,CITY,STATE,ZIPCODE,SEX,AGE,DRIVERS_LICENSE_STATE,SAFETY_EQUIPMENT,AIRBAG_DEPLOYED,INJURY_CLASSIFICATION,DRIVER_ACTION,PHYSICAL_CONDITION
0,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,O749947,DRIVER,834816.0,CHICAGO,IL,60651,M,25.0,IL,NONE PRESENT,DEPLOYMENT UNKNOWN,NO INDICATION OF INJURY,UNKNOWN,UNKNOWN
1,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,O871921,DRIVER,827212.0,CHICAGO,IL,60620,M,37.0,IL,SAFETY BELT USED,DID NOT DEPLOY,NO INDICATION OF INJURY,NONE,NORMAL
2,71162af7bf22799b776547132ebf134b5b438dcf3dac6b...,O10018,DRIVER,9579.0,NaN,NaN,NaN,X,NaN,NaN,USAGE UNKNOWN,DEPLOYMENT UNKNOWN,NO INDICATION OF INJURY,IMPROPER BACKING,UNKNOWN
3,c21c476e2ccc41af550b5d858d22aaac4ffc88745a1700...,O10038,DRIVER,9598.0,NaN,NaN,NaN,X,NaN,NaN,USAGE UNKNOWN,DEPLOYMENT UNKNOWN,NO INDICATION OF INJURY,UNKNOWN,UNKNOWN
4,eb390a4c8e114c69488f5fb8a097fe629f5a92fd528cf4...,O10039,DRIVER,9600.0,NaN,NaN,NaN,X,NaN,NaN,USAGE UNKNOWN,DEPLOYMENT UNKNOWN,NO INDICATION OF INJURY,UNKNOWN,UNKNOWN
5,dd1bce4bd6d0be4c247714dcabab44e6563c62b913229b...,O10041,DRIVER,9601.0,NaN,NaN,NaN,X,NaN,NaN,USAGE UNKNOWN,DEPLOYMENT UNKNOWN,NO INDICATION OF INJURY,UNKNOWN,UNKNOWN
6,4bd2ee6bb306902b99a9c2ae55cf4fcffec00879e39759...,O10062,DRIVER,9621.0,NaN,NaN,NaN,X,NaN,XX,USAGE UNKNOWN,DEPLOYMENT UNKNOWN,NO INDICATION OF INJURY,UNKNOWN,UNKNOWN
7,9c03b6fcc6d72cf3ee2cb9ea754ea7e4617ab965142552...,O10066,DRIVER,9623.0,NaN,NaN,NaN,X,NaN,XX,USAGE UNKNOWN,DEPLOYMENT UNKNOWN,NO INDICATION OF INJURY,UNKNOWN,UNKNOWN
8,2e7e0c1682100a200a46c25a532eda4febb00a2ae49ea9...,O1007,DRIVER,958.0,NaN,NaN,NaN,X,NaN,NaN,USAGE UNKNOWN,DEPLOYMENT UNKNOWN,NO INDICATION OF INJURY,UNKNOWN,UNKNOWN
9,8bf8069b3d839b732fd35d2c9d8caddeddbb4b8978e84d...,O10088,DRIVER,9645.0,NaN,NaN,NaN,X,NaN,NaN,USAGE UNKNOWN,DEPLOYMENT UNKNOWN,NO INDICATION OF INJURY,UNKNOWN,UNKNOWN


In [74]:
df_people_driver.describe()

,VEHICLE_ID,AGE
count,1.117315e+06,818652.000000
mean,6.738542e+05,40.013475
std,3.924537e+05,15.836726
min,2.000000e+00,-177.000000
25%,3.338875e+05,27.000000
50%,6.705940e+05,37.000000
75%,1.014066e+06,51.000000
max,1.358762e+06,110.000000


In [75]:
# Dropping rows with driver age bigger than 90 and below 18
df_people_driver_age = df_people_driver[(df_people_driver['AGE'] >= 18) & (df_people_driver['AGE'] <= 90) | (df_people_driver['AGE'].isna())]

In [76]:
# Replacing all UNKNOWN values to missing values
#Unknown = ['UNKNOWN','USAGE UNKNOWN', 'DEPLOYMENT UNKNOWN']
df_vehicles_w_occ.replace({'Unknown' :np.nan , 'USAGE UNKNOWN' :np.nan , 'DEPLOYMENT UNKNOWN':np.nan }, inplace=True)

C:\Users\milad\Documents\Flatiron\Anaconda\envs\learn-env\lib\site-packages\pandas\core\frame.py:4379: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [77]:
merged = df_crashes_drop.merge(df_vehicles_w_occ, on='CRASH_RECORD_ID')
print(merged.shape)

(1160337, 28)


In [78]:
merged[merged.duplicated(subset = 'VEHICLE_ID') == True].shape


(30260, 28)

In [79]:
df = merged.merge(df_people_driver_age, on=['VEHICLE_ID','CRASH_RECORD_ID'])
print(df.shape)

(1099817, 41)


In [80]:
df.head()

,CRASH_RECORD_ID,RD_NO,CRASH_DATE,POSTED_SPEED_LIMIT,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,ROADWAY_SURFACE_COND,ROAD_DEFECT,CRASH_TYPE,DAMAGE,PRIM_CONTRIBUTORY_CAUSE,STREET_NAME,NUM_UNITS,INJURIES_TOTAL,INJURIES_FATAL,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,VEHICLE_ID,MAKE,MODEL,LIC_PLATE_STATE,VEHICLE_YEAR,TRAVEL_DIRECTION,OCCUPANT_CNT,PERSON_ID,PERSON_TYPE,CITY,STATE,ZIPCODE,SEX,AGE,DRIVERS_LICENSE_STATE,SAFETY_EQUIPMENT,AIRBAG_DEPLOYED,INJURY_CLASSIFICATION,DRIVER_ACTION,PHYSICAL_CONDITION
0,062f5a6f6b87b762165d4da04d6d3a181385776a10b051...,JF378246,08/31/2022 10:13:00 PM,30,CLEAR,"DARKNESS, LIGHTED ROAD",SIDESWIPE SAME DIRECTION,DRY,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH,"OVER $1,500",UNABLE TO DETERMINE,CICERO AVE,4,2.0,0.0,22,4,8,41.959389,-87.747348,1340131.0,NaN,OTHER (EXPLAIN IN NARRATIVE),NaN,NaN,N,1.0,O1410268,DRIVER,NaN,NaN,NaN,X,NaN,NaN,USAGE UNKNOWN,DEPLOYMENT UNKNOWN,NO INDICATION OF INJURY,UNKNOWN,UNKNOWN
1,062f5a6f6b87b762165d4da04d6d3a181385776a10b051...,JF378246,08/31/2022 10:13:00 PM,30,CLEAR,"DARKNESS, LIGHTED ROAD",SIDESWIPE SAME DIRECTION,DRY,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH,"OVER $1,500",UNABLE TO DETERMINE,CICERO AVE,4,2.0,0.0,22,4,8,41.959389,-87.747348,1340137.0,LINCOLN,TOWNCAR,IL,1994.0,N,2.0,O1410269,DRIVER,EVANSTON,IL,60201,M,67.0,IL,USAGE UNKNOWN,"DEPLOYED, COMBINATION",NONINCAPACITATING INJURY,NONE,NORMAL
2,0115ade9a755e835255508463f7e9c4a9a0b47e9304238...,JF318029,07/15/2022 12:45:00 AM,30,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,DRY,NaN,NO INJURY / DRIVE AWAY,"OVER $1,500",UNABLE TO DETERMINE,CENTRAL PARK AVE,2,0.0,0.0,0,6,7,41.886336,-87.716203,1310014.0,HYUNDAI,OTHER (EXPLAIN IN NARRATIVE),NaN,NaN,W,1.0,O1378702,DRIVER,NaN,NaN,NaN,X,NaN,NaN,USAGE UNKNOWN,DEPLOYMENT UNKNOWN,NO INDICATION OF INJURY,UNKNOWN,UNKNOWN
3,0115ade9a755e835255508463f7e9c4a9a0b47e9304238...,JF318029,07/15/2022 12:45:00 AM,30,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,DRY,NaN,NO INJURY / DRIVE AWAY,"OVER $1,500",UNABLE TO DETERMINE,CENTRAL PARK AVE,2,0.0,0.0,0,6,7,41.886336,-87.716203,1310017.0,GENERAL MOTORS CORPORATION (GMC),OTHER (EXPLAIN IN NARRATIVE),IL,2015.0,N,1.0,O1378703,DRIVER,CHIACGO,IL,60649,M,62.0,IL,USAGE UNKNOWN,NOT APPLICABLE,NO INDICATION OF INJURY,UNKNOWN,NORMAL
4,017040c61958d2fa977c956b2bd2d6759ef7754496dc96...,JF324552,07/15/2022 06:50:00 PM,30,CLEAR,DAYLIGHT,REAR END,DRY,NO DEFECTS,NO INJURY / DRIVE AWAY,"OVER $1,500",UNABLE TO DETERMINE,ASHLAND AVE,2,0.0,0.0,18,6,7,41.925111,-87.667997,1313322.0,DODGE,JOURNEY,IL,2017.0,S,1.0,O1382153,DRIVER,CHICAGO,IL,60644,F,58.0,IL,USAGE UNKNOWN,DID NOT DEPLOY,NO INDICATION OF INJURY,UNKNOWN,UNKNOWN


In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1099817 entries, 0 to 1099816
Data columns (total 41 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   CRASH_RECORD_ID          1099817 non-null  object 
 1   RD_NO                    1092150 non-null  object 
 2   CRASH_DATE               1099817 non-null  object 
 3   POSTED_SPEED_LIMIT       1099817 non-null  int64  
 4   WEATHER_CONDITION        1055149 non-null  object 
 5   LIGHTING_CONDITION       1065383 non-null  object 
 6   FIRST_CRASH_TYPE         1099817 non-null  object 
 7   ROADWAY_SURFACE_COND     1025920 non-null  object 
 8   ROAD_DEFECT              926974 non-null   object 
 9   CRASH_TYPE               1099817 non-null  object 
 10  DAMAGE                   1099817 non-null  object 
 11  PRIM_CONTRIBUTORY_CAUSE  1099817 non-null  object 
 12  STREET_NAME              1099814 non-null  object 
 13  NUM_UNITS                1099817 non-null 

In [82]:
# creating bins for times 
# 0-6 = Late Night/Early Morning 
# 6-12 = Morning 
# 12-18 = Afternoon/Rush Hour
# 18-23 = Late Evening 
df['time_bins'] = pd.cut(x=df['CRASH_HOUR'], bins = [0,6,12,18,23], 
                         labels = ['Late Night/Early Morning', 
                        'Morning', 'Afternoon/Rush Hour','Late Evening'])
df.head()

,CRASH_RECORD_ID,RD_NO,CRASH_DATE,POSTED_SPEED_LIMIT,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,ROADWAY_SURFACE_COND,ROAD_DEFECT,CRASH_TYPE,DAMAGE,PRIM_CONTRIBUTORY_CAUSE,STREET_NAME,NUM_UNITS,INJURIES_TOTAL,INJURIES_FATAL,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,VEHICLE_ID,MAKE,MODEL,LIC_PLATE_STATE,VEHICLE_YEAR,TRAVEL_DIRECTION,OCCUPANT_CNT,PERSON_ID,PERSON_TYPE,CITY,STATE,ZIPCODE,SEX,AGE,DRIVERS_LICENSE_STATE,SAFETY_EQUIPMENT,AIRBAG_DEPLOYED,INJURY_CLASSIFICATION,DRIVER_ACTION,PHYSICAL_CONDITION,time_bins
0,062f5a6f6b87b762165d4da04d6d3a181385776a10b051...,JF378246,08/31/2022 10:13:00 PM,30,CLEAR,"DARKNESS, LIGHTED ROAD",SIDESWIPE SAME DIRECTION,DRY,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH,"OVER $1,500",UNABLE TO DETERMINE,CICERO AVE,4,2.0,0.0,22,4,8,41.959389,-87.747348,1340131.0,NaN,OTHER (EXPLAIN IN NARRATIVE),NaN,NaN,N,1.0,O1410268,DRIVER,NaN,NaN,NaN,X,NaN,NaN,USAGE UNKNOWN,DEPLOYMENT UNKNOWN,NO INDICATION OF INJURY,UNKNOWN,UNKNOWN,Late Evening
1,062f5a6f6b87b762165d4da04d6d3a181385776a10b051...,JF378246,08/31/2022 10:13:00 PM,30,CLEAR,"DARKNESS, LIGHTED ROAD",SIDESWIPE SAME DIRECTION,DRY,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH,"OVER $1,500",UNABLE TO DETERMINE,CICERO AVE,4,2.0,0.0,22,4,8,41.959389,-87.747348,1340137.0,LINCOLN,TOWNCAR,IL,1994.0,N,2.0,O1410269,DRIVER,EVANSTON,IL,60201,M,67.0,IL,USAGE UNKNOWN,"DEPLOYED, COMBINATION",NONINCAPACITATING INJURY,NONE,NORMAL,Late Evening
2,0115ade9a755e835255508463f7e9c4a9a0b47e9304238...,JF318029,07/15/2022 12:45:00 AM,30,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,DRY,NaN,NO INJURY / DRIVE AWAY,"OVER $1,500",UNABLE TO DETERMINE,CENTRAL PARK AVE,2,0.0,0.0,0,6,7,41.886336,-87.716203,1310014.0,HYUNDAI,OTHER (EXPLAIN IN NARRATIVE),NaN,NaN,W,1.0,O1378702,DRIVER,NaN,NaN,NaN,X,NaN,NaN,USAGE UNKNOWN,DEPLOYMENT UNKNOWN,NO INDICATION OF INJURY,UNKNOWN,UNKNOWN,NaN
3,0115ade9a755e835255508463f7e9c4a9a0b47e9304238...,JF318029,07/15/2022 12:45:00 AM,30,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,DRY,NaN,NO INJURY / DRIVE AWAY,"OVER $1,500",UNABLE TO DETERMINE,CENTRAL PARK AVE,2,0.0,0.0,0,6,7,41.886336,-87.716203,1310017.0,GENERAL MOTORS CORPORATION (GMC),OTHER (EXPLAIN IN NARRATIVE),IL,2015.0,N,1.0,O1378703,DRIVER,CHIACGO,IL,60649,M,62.0,IL,USAGE UNKNOWN,NOT APPLICABLE,NO INDICATION OF INJURY,UNKNOWN,NORMAL,NaN
4,017040c61958d2fa977c956b2bd2d6759ef7754496dc96...,JF324552,07/15/2022 06:50:00 PM,30,CLEAR,DAYLIGHT,REAR END,DRY,NO DEFECTS,NO INJURY / DRIVE AWAY,"OVER $1,500",UNABLE TO DETERMINE,ASHLAND AVE,2,0.0,0.0,18,6,7,41.925111,-87.667997,1313322.0,DODGE,JOURNEY,IL,2017.0,S,1.0,O1382153,DRIVER,CHICAGO,IL,60644,F,58.0,IL,USAGE UNKNOWN,DID NOT DEPLOY,NO INDICATION OF INJURY,UNKNOWN,UNKNOWN,Afternoon/Rush Hour


In [83]:
df.CRASH_TYPE.value_counts()

NO INJURY / DRIVE AWAY              814365
INJURY AND / OR TOW DUE TO CRASH    285452
Name: CRASH_TYPE, dtype: int64

In [84]:
# Dropping Unnecassary Features
df_relv = df.drop([
    'CRASH_RECORD_ID',
    'RD_NO',
    'PERSON_ID',
    'CRASH_DATE',
    'VEHICLE_ID',
    'CITY',
    'STATE',
    'ZIPCODE',
    'LIC_PLATE_STATE',
    'TRAVEL_DIRECTION', 
    'DRIVERS_LICENSE_STATE',
    'INJURY_CLASSIFICATION',
    'DRIVER_ACTION',
    'PHYSICAL_CONDITION'],
axis=1)

In [85]:
df_relv.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1099817 entries, 0 to 1099816
Data columns (total 28 columns):
 #   Column                   Non-Null Count    Dtype   
---  ------                   --------------    -----   
 0   POSTED_SPEED_LIMIT       1099817 non-null  int64   
 1   WEATHER_CONDITION        1055149 non-null  object  
 2   LIGHTING_CONDITION       1065383 non-null  object  
 3   FIRST_CRASH_TYPE         1099817 non-null  object  
 4   ROADWAY_SURFACE_COND     1025920 non-null  object  
 5   ROAD_DEFECT              926974 non-null   object  
 6   CRASH_TYPE               1099817 non-null  object  
 7   DAMAGE                   1099817 non-null  object  
 8   PRIM_CONTRIBUTORY_CAUSE  1099817 non-null  object  
 9   STREET_NAME              1099814 non-null  object  
 10  NUM_UNITS                1099817 non-null  int64   
 11  INJURIES_TOTAL           1099817 non-null  float64 
 12  INJURIES_FATAL           1099817 non-null  float64 
 13  CRASH_HOUR               10

In [86]:
# Splitting features into numeric and categorical
numeric_columns = [
    'POSTED_SPEED_LIMIT',
    'NUM_UNITS',
    'VEHICLE_YEAR',
    'OCCUPANT_CNT',
    'AGE',
    
]

cat_columns = [
    'WEATHER_CONDITION',  
    'LIGHTING_CONDITION',       
    'FIRST_CRASH_TYPE'   ,       
    'ROADWAY_SURFACE_COND',      
    'ROAD_DEFECT',   
    'CRASH_TYPE',   
    'DAMAGE', 
    'PRIM_CONTRIBUTORY_CAUSE', 
    'STREET_NAME',              
#    'NUM_UNITS',                 
#    'INJURIES_TOTAL',           
#    'INJURIES_FATAL',          
    'CRASH_HOUR',                
    'CRASH_DAY_OF_WEEK',        
    'CRASH_MONTH',                
    'LATITUDE',                
    'LONGITUDE',               
    'MAKE',                      
    'MODEL',                    
#    'VEHICLE_YEAR',            
#    'OCCUPANT_CNT',            
    'PERSON_TYPE',               
    'SEX',                      
#    'AGE',                      
    'SAFETY_EQUIPMENT',          
    'AIRBAG_DEPLOYED',           
    'time_bins',
]


Target = df[[
    'INJURIES_TOTAL',           
#    'INJURIES_FATAL',    
]]

numeric_df = df[numeric_columns]
cat_df = df[cat_columns]

# Train_Test Split

In [87]:
X = pd.concat([numeric_df,cat_df] , axis = 1)
Y = Target

X_train, X_test, y_train, y_test = train_test_split(X, Target, test_size = .25, random_state = 42)

X_train_nums = X_train [[
    'POSTED_SPEED_LIMIT',
    'NUM_UNITS',
    'VEHICLE_YEAR',
    'OCCUPANT_CNT',
    'AGE',
    
]]

X_train_cats = X_train [[
    'WEATHER_CONDITION',  
#    'LIGHTING_CONDITION',       
#    'FIRST_CRASH_TYPE'   ,       
#    'ROADWAY_SURFACE_COND',      
#    'ROAD_DEFECT',   
#    'CRASH_TYPE',   
#    'DAMAGE', 
#    'PRIM_CONTRIBUTORY_CAUSE', 
#    'STREET_NAME',              
#    'NUM_UNITS',                 
#    'INJURIES_TOTAL',           
#    'INJURIES_FATAL',          
#    'CRASH_HOUR',                
#    'CRASH_DAY_OF_WEEK',        
#    'CRASH_MONTH',                
#    'LATITUDE',                
#    'LONGITUDE',               
#    'MAKE',                      
#    'MODEL',                    
#    'VEHICLE_YEAR',            
#    'OCCUPANT_CNT',            
#    'PERSON_TYPE',               
#    'SEX',                      
#    'AGE',                      
#    'SAFETY_EQUIPMENT',          
#   'AIRBAG_DEPLOYED',           
#   'time_bins',
]]

X_train = pd.concat([X_train_nums,X_train_cats] , axis = 1)

In [ ]:
X_train_cats[:50]

In [ ]:
imputer = SimpleImputer(strategy="most_frequent", verbose = True) 
X_train_filled = imputer.fit_transform(X_train_cats)

In [ ]:
XX = pd.DataFrame(X_train_filled, columns = 'WEATHER_CONDITION', index = X_train_cats.index)

# Impute and Scale Data Using Pipeline

In [ ]:
#numerical_imputer_pipeline = Pipeline(steps=[
#    ('ImN', SimpleImputer(strategy="median"))
#])    
#categorical_imputer_pipeline = Pipeline(steps=[
#    ('ImC', SimpleImputer(strategy="most_frequent"))  
#])
numerical_pipeline = Pipeline(steps=[
    ('ImN', SimpleImputer(strategy="median")),
    ('ss', StandardScaler())
])
                
categorical_pipeline = Pipeline(steps=[
    ('ImC', SimpleImputer(strategy="most_frequent")),
#    ('ohe', OneHotEncoder(drop='first',
#                         sparse=False))
])

trans = ColumnTransformer(transformers=[
#    ('numerical_I', numerical_pipeline, X_train_nums.columns),
#    ('categorical_I', categorical_imputer_pipeline, X_train_cats.columns),
    ('numerical', numerical_pipeline, X_train_nums.columns),
    ('categorical', categorical_pipeline, X_train_cats.columns)
])

In [ ]:
prep_pipe = Pipeline(steps=[
    ('trans', trans)
])

In [ ]:
prep_pipe.fit(X_train)